### This Notebook scrapes all tracks and saves the pos data into a parquet-file

In [86]:
import fastf1 as ff1
import numpy as np
import pandas as pd
from tqdm import tqdm

Get all GPs from the years 2018 to 2024

In [87]:
all_gps = []
for year in range(2018,2025):
    temp_df = pd.DataFrame(ff1.get_event_schedule(year)["EventName"])
    temp_df["year"] = year
    all_gps.append(temp_df)




df_all_gps = pd.concat(all_gps)

Drop testing events

In [88]:
df_all_gps = df_all_gps.replace({"Pre-Season":None},regex=True)
df_all_gps = df_all_gps.dropna()

Scrape the track data for every track

In [89]:
# Hilfsfunktion um Positionen gleich richtig zu rotieren
def rotate(xy, *, angle):
    rot_mat = np.array(
        [[np.cos(angle), np.sin(angle)], [-np.sin(angle), np.cos(angle)]]
    )
    return np.matmul(xy, rot_mat)

In [90]:
all_track_data = []
for _,gp_data in tqdm(df_all_gps.iterrows()):
    year = gp_data["year"]
    gp = gp_data["EventName"]

    session = ff1.get_session(year,gp,"Q")
    session.load()
    df_temp_pos_data = session.laps.pick_fastest().get_pos_data()
    # add the first row to end to make the lap complete
    temp_first_row = df_temp_pos_data.iloc[:1,:]
    df_temp_pos_data = pd.concat([df_temp_pos_data,temp_first_row])
    
    track = df_temp_pos_data.loc[:, ("X", "Y")].to_numpy()

    circuit_info = session.get_circuit_info()
    # Convert the rotation angle from degrees to radian.
    track_angle = circuit_info.rotation / 180 * np.pi

    # Rotate and plot the track map.
    rotated_track = rotate(track, angle=track_angle)
    
    df_temp_track_data = pd.DataFrame(rotated_track, columns=["x", "y"])
    
    df_temp_track_data["event_name"] = gp
    df_temp_track_data["year"] = year
    all_track_data.append(df_temp_track_data)

df_track_data = pd.concat(all_track_data)



0it [00:00, ?it/s]core           INFO 	Loading data for Australian Grand Prix - Qualifying [v3.3.7]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['44', '7', '5', '33', '3', '20', '8', '27', '55', '77', '14', '2', '11', '18', '31', '28', '9', '16', '35', '10']
core        WARNING 	Failed to preserve data type for column 'X' while merging telemetry.


KeyError: 'DriverNumber'

In [92]:
df_track_data = pd.concat(all_track_data)


Save to parquet

In [96]:
df_track_data.to_parquet("./static/data/all_tracks.parquet",index=False)